## Prepare dataset for BERT

The paths utilized are defiend in the `path_module.py`

### 1. Import libraries

In [1]:
from modules.bert_module import *
from modules.path_module import *
import os
import joblib
from modules.lipcot_module import LiPCoTdata
from transformers import BertTokenizer
from hugtokencraft import editor # type: ignore

### 2. Load tokenized data

In [2]:
train_data=joblib.load(os.path.join(tokenized_data_savepath,"train_data.joblib"))
test_data=joblib.load(os.path.join(tokenized_data_savepath,"test_data.joblib"))
val_data=joblib.load(os.path.join(tokenized_data_savepath,"val_data.joblib"))

lipcot_model=joblib.load(os.path.join(lipcot_model_savepath,"lipcot_model.joblib"))
tokenizer=editor.load_tokenizer(BertTokenizer,lipcot_model['tokenizer_path'])


Tokenization model Validation: Passed

Tokenizer loaded with vocabulary size: 69


### 3. Prepare data for BERT
We extract only the tokenized text and labels from dataset for BERT

In [3]:
train_text,train_label=prepare_bert_data(train_data)
val_text,val_label=prepare_bert_data(val_data)
test_text,test_label=prepare_bert_data(test_data)

Let's inspect tokenization padding and attention masks

In [4]:
#%% check tokenization padding and attention masks

sentence1 = train_text[0]
sentence2 = train_text[1]
padded_sequences = tokenizer([sentence1, sentence2], padding=True)
print("input_ids")
print(len(padded_sequences["input_ids"][0]))
print(len(padded_sequences["input_ids"][1]))
print(padded_sequences["input_ids"][0])
print(padded_sequences["input_ids"][1])

print("token_type_ids")
print(len(padded_sequences["token_type_ids"][0]))
print(len(padded_sequences["token_type_ids"][1]))
print(padded_sequences["token_type_ids"][0])
print(padded_sequences["token_type_ids"][1])

print("attention_mask")
print(len(padded_sequences["attention_mask"][0]))
print(len(padded_sequences["attention_mask"][1]))
print(padded_sequences["attention_mask"][0])
print(padded_sequences["attention_mask"][1])

# this is what happens for block_size=5 in LikeByLineTextDataset
padded_sequences = tokenizer.batch_encode_plus([sentence1, sentence2],max_length=tokenizer.model_max_length,
                                               padding='max_length', truncation=True, add_special_tokens=True)
print("input_ids")
print(padded_sequences["input_ids"])
print("token_type_ids")
print(padded_sequences["token_type_ids"])
print("attention_mask")
print(padded_sequences["attention_mask"])


input_ids
61
61
[2, 49, 51, 13, 13, 13, 31, 31, 41, 56, 50, 20, 22, 56, 18, 43, 19, 43, 43, 32, 8, 20, 10, 50, 48, 56, 64, 64, 51, 10, 31, 51, 31, 13, 13, 31, 59, 51, 10, 41, 65, 51, 56, 30, 30, 68, 47, 19, 47, 63, 22, 50, 10, 22, 56, 19, 64, 47, 51, 14, 3]
[2, 67, 30, 49, 23, 61, 20, 23, 61, 14, 63, 20, 50, 63, 22, 40, 22, 20, 50, 56, 8, 8, 56, 10, 13, 37, 10, 10, 18, 23, 49, 32, 8, 23, 35, 23, 17, 59, 61, 44, 50, 59, 50, 51, 59, 63, 63, 56, 63, 63, 58, 56, 46, 40, 10, 46, 66, 56, 18, 50, 3]
token_type_ids
61
61
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask
61
61
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

### 4. Prepare dataset for BERT

In [5]:
max_sentence_length=lipcot_model['model_max_length']-2
all_sentence_lengths=[max_sentence_length]
train_dataset = BertDataset(train_text, train_label,tokenizer,max_length=max_sentence_length+2)
val_dataset = BertDataset(val_text, val_label,tokenizer,max_length=max_sentence_length+2)
test_dataset = BertDataset(test_text, test_label,tokenizer,max_length=max_sentence_length+2)

Let's check the attention mask

In [6]:
gg=train_dataset.__getitem__(1)
print(gg['attention_mask'])
print(gg['attention_mask'].shape)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([128])


### 5. Save the datasets

In [7]:
joblib.dump(train_dataset, os.path.join(bert_datapath,"train_data.joblib"))
joblib.dump(val_dataset, os.path.join(bert_datapath,"val_data.joblib"))
joblib.dump(test_dataset, os.path.join(bert_datapath,"test_data.joblib"))

['c:\\Users\\fahim\\Documents\\MyBertProject\\data/forBert\\test_data.joblib']